In [1]:
%matplotlib inline
from PIL import Image
import pandas as pd
import seaborn as sns
from sklearn.linear_model import LogisticRegression
import os
import itertools
from tqdm import *
from tqdm import tqdm_notebook as tqdm
import random
import numpy as np
import math

from IPython.display import clear_output

# Step 1: Load Data Set

In [189]:
fileNames = []
for file in os.listdir("data/train_1"):
    if file.endswith(".jpg"):
        fileNames.append(file)
# fileNamesDF = pd.DataFrame(fileNames)

In [201]:
#only take certain artist so that the probabilities aren't basically all false when doing the tupples
minNumPaintingsPerArtist = 200

# #for each artist, only take n of their paintings
numPaintingsPerAuthor = 100


trainInfo = pd.read_csv('data/train_info.csv')
mostPopularArtists = trainInfo['artist'].value_counts()[trainInfo['artist'].value_counts() > minNumPaintingsPerArtist]

trainInfo = trainInfo[trainInfo['filename'].isin(fileNames)]
fileNamesDFAll = trainInfo[trainInfo['artist'].isin(mostPopularArtists.index)]


fileNamesDF = pd.DataFrame()
for artist in tqdm(mostPopularArtists.index):
    fileNamesDF = fileNamesDF.append(trainInfo[trainInfo['artist'] == artist][:numPaintingsPerAuthor])
    
fileNamesDF = fileNamesDF['filename']

In [202]:
# Load the all_data_info for the dataset we are using
allInfo = pd.read_csv('data/all_data_info.csv')
allInfo = allInfo[allInfo['new_filename'].isin(fileNamesDF)]

In [204]:
featuresDF = pd.DataFrame(allInfo['new_filename'])
featuresDF['pixelsx'] = allInfo['pixelsx']
featuresDF['pixelsy'] = allInfo['pixelsy']

In [205]:
def normalizeSeries(series, maxNum):
    series = series/maxNum
    return series

In [206]:
def featurizeImage(filename):
    # Extract the features from the actual image
    jpgfile = Image.open("data/train_1/" + filename)
#     print(jpgfile)
    
    bands = jpgfile.getbands()
    if bands[0] == 'L':
        # grayscale image
        imageDF = pd.DataFrame(list(jpgfile.getdata()),columns=['grayscale'])
#         imageDF['grayscale'] = normalizeSeries(imageDF['grayscale',])
    elif len(bands) == 4:
        #get red, green and blue chanels 
        imageDF = pd.DataFrame(list(jpgfile.getdata()),columns=['red','green','blue','alpha'])
        imageDF['red'] = normalizeSeries(imageDF['red'],255)
        imageDF['green'] = normalizeSeries(imageDF['green'],255)
        imageDF['blue'] = normalizeSeries(imageDF['blue'],255)  
        imageDF['alpha'] = normalizeSeries(imageDF['alpha'],255)  
    elif len(bands) == 3:
        #get red, green and blue chanels 
        imageDF = pd.DataFrame(list(jpgfile.getdata()),columns=['red','green','blue'])
        imageDF['red'] = normalizeSeries(imageDF['red'],255)
        imageDF['green'] = normalizeSeries(imageDF['green'],255)
        imageDF['blue'] = normalizeSeries(imageDF['blue'],255)  
    else:
        imageDF = pd.DataFrame(list(jpgfile.getdata()))
        print(bands)
        return imageDF, bands

    return imageDF.mean().values

In [207]:
featuresDF.index

Int64Index([   511,    697,    773,    867,    921,    933,    995,   1277,
              1332,   1546,
            ...
            101294, 101629, 101725, 102085, 102113, 102402, 102565, 102862,
            103030, 103049],
           dtype='int64', length=1661)

In [208]:
imageFeaturesDF = pd.DataFrame(index=featuresDF.index,columns=['features']) 
print(imageFeaturesDF.index)
for row in tqdm(list(featuresDF.index)):
    imageFeatures = featurizeImage(featuresDF.loc[row]['new_filename'])
    imageFeaturesDF.loc[row]['features'] = imageFeatures


Int64Index([   511,    697,    773,    867,    921,    933,    995,   1277,
              1332,   1546,
            ...
            101294, 101629, 101725, 102085, 102113, 102402, 102565, 102862,
            103030, 103049],
           dtype='int64', length=1661)


In [209]:
imageFeaturesDF.head()

,features
511,"[0.5428758332157436, 0.567912843558902, 0.4950..."
697,"[0.5401178080546106, 0.5401178080546106, 0.540..."
773,[109.78050646073648]
867,"[0.498340133186191, 0.498340133186191, 0.49834..."
921,"[0.6727659245903405, 0.6727659245903405, 0.672..."


In [210]:
imageFeaturesDF.to_csv('image_features_extracted2.csv')

In [211]:
featuresDF['imgFeatures'] = imageFeaturesDF['features']

In [212]:
featuresDF.head()

,new_filename,pixelsx,pixelsy,imgFeatures
511,13211.jpg,4340.0,2952.0,"[0.5428758332157436, 0.567912843558902, 0.4950..."
697,14108.jpg,3899.0,2874.0,"[0.5401178080546106, 0.5401178080546106, 0.540..."
773,1155.jpg,3899.0,2732.0,[109.78050646073648]
867,13673.jpg,3887.0,2601.0,"[0.498340133186191, 0.498340133186191, 0.49834..."
921,16144.jpg,3905.0,2538.0,"[0.6727659245903405, 0.6727659245903405, 0.672..."


In [302]:
def compareImages(img1, img2):
    
    diffx = np.abs(img1['pixelsx']-img2['pixelsx']) / (img1['pixelsx'] + img2['pixelsx'])
    diffy = np.abs(img1['pixelsy']-img2['pixelsy']) / (img1['pixelsy'] + img2['pixelsy'])
    
    diffSize = (diffx + diffy)/2
    
    img1Features = img1['imgFeatures']
    img2Features = img2['imgFeatures']
    
    imgFeatureScore = 0
    
    if (len(img1Features) == len(img2Features)):
        imgFeatureDiffs = np.abs(img1Features - img2Features)
        imgFeatureScore = sum(imgFeatureDiffs)

    score = (diffSize+imgFeatureScore)/2
    return 1-score

In [328]:
nTrain = 100
nDev = 100
nTest = 10

featuresTrain = featuresDF.sample(n=nTrain)
# print(featuresTrain.shape)
featuresDev = featuresDF.drop(featuresTrain.index).sample(n=nDev)
# print(featuresDev.shape)
featuresTest = featuresDF.drop(featuresDev.index).drop(featuresTrain.index).sample(n=nTest)
# print(featuresTest.shape)

In [356]:
# the score represents how similar two images are
scores = []
iloc1 = featuresTrain.sample(frac=0.5).index
iloc2 = featuresTrain.sample(frac=0.5).index

# print(len(iloc2))
# print(nTrials)

for i, loc in tqdm(enumerate(iloc1)):
    scores.append(compareImages(featuresTrain.loc[loc],featuresTrain.loc[iloc2[i]]))

filenames1 = featuresTrain.loc[iloc1]['new_filename'].values
filenames2 = featuresTrain.loc[iloc2]['new_filename'].values

truth = trainInfo[trainInfo['filename'].isin(filenames1)]['artist'].values == trainInfo[trainInfo['filename'].isin(filenames2)]['artist'].values

In [357]:
truth

array([ True, False, False, False, False, False, False, False, False,
       False,  True, False, False,  True, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False,  True, False, False, False, False,  True,  True, False,
       False, False, False, False, False, False, False, False, False,
        True, False, False, False, False])

In [342]:
probThreshold = 0.5
def generatePrediction(score,threshold):
    if score > threshold:
        return True
    else:
        return False
    
def generatePredictions(scores,threshold):
    predictions = []
    for curScore in tqdm(scores):
        predictions.append(generatePrediction(curScore,threshold))
    
    return predictions


generatePredictions(scores,probThreshold)

[True,
 True,
 True,
 False,
 True,
 True,
 True,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 False,
 True,
 True,
 True,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 False,
 False,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 True,
 False,
 True,
 True,
 True,
 True,
 False,
 True,
 False]

In [ ]:
solutionDF = 